## Analysing the data from the Market Research Survey

In [1]:
# import libraries
import pandas as pd

In [5]:
# import csv file as dataframe
data_path = "tudublin_amenities_access_survey.csv"
survey_data = pd.read_csv(data_path, delimiter=",", encoding='unicode_escape')
survey_data.head(5)

,Id,Start time,Completion time,Email,Name,Which county do you work/live in?,Which sector do you work in?\n,Does your job involve accessing/working with data on publicly available amenities?\n,Which devices do you primarily use for your work?\n,What type of public amenities information do you usually require for your job?\n,...,Do you have any additional suggestions/feedback on our proposal ?\n,"If you'd like to contact us for more information on the project, please share your email below\n",Which devices do you primarily use for your day-to-day personal tasks?\n,How often do you use digital tools/applications for navigation?\n,"Based on the demo below, how useful would a web app be for accessing parking locations and quantities in your day-to-day?\n",Why would this be impractical for you?\n,Please tell us what you are using instead\n,What other public amenity would you like more information on?,What other features would you like to see alongside location & quantity of parking spaces?\n,"If you'd like to contact us for more information on the project, please share your email below\n1"
0,1,22/10/2024 18:14,22/10/2024 18:17,anonymous,NaN,NaN,Education,Yes,Desktop computer;Laptop;Smartphone;,"Mechanical (water grid, electric grid, etc);Tr...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,29/10/2024 16:29,29/10/2024 16:36,anonymous,NaN,Dublin,Government,No,NaN,NaN,...,NaN,NaN,Desktop computer;Laptop;Smartphone;,Weekly,Extremely useful,NaN,NaN,NaN,Cost;Parking hours;,NaN
2,3,31/10/2024 12:18,31/10/2024 12:19,anonymous,NaN,Fingal,Construction,No,NaN,NaN,...,NaN,NaN,Desktop computer;,Never,Somewhat useful,NaN,NaN,NaN,Parking hours;,NaN
3,4,31/10/2024 12:25,31/10/2024 12:29,anonymous,NaN,Dublin,Architecture,Yes,Desktop computer;Laptop;Smartphone;,"Recreational (parks, sport facilities, hiking ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,31/10/2024 12:24,31/10/2024 12:29,anonymous,NaN,Dublin,Construction,Yes,Desktop computer;,"Healthcare & Safety (emergency services, hospi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
